### This notebook deals with all things Multi Class classification. Starting with basic NN

In [1]:
import torch
from torch import nn
from torch.nn import functional as F 
from torch import optim

In [9]:
n = torch.randn((1, 4))
n

tensor([[ 0.5382, -1.1093,  2.0243,  1.7545]])

In [10]:
smax = nn.Softmax(dim=1)  # the function exists between 0 to 1 like probability, so we use it
print(smax(n))
smax(n).argmax()  # use argmax to get the index of max numbers

tensor([[0.1113, 0.0214, 0.4918, 0.3755]])


tensor(2)

In [70]:
# Defining a simple NN model

class MulClassify(nn.Module):
    def __init__(self, in_features, out_features, hidden_features):
        super(MulClassify, self).__init__()
        self.lin1 = nn.Linear(in_features, hidden_features)  # first layer connected to hidden
        self.relu = nn.ReLU()
        self.lin2 = nn.Linear(hidden_features, out_features)  # hidden to out features
        # self.soft = nn.Softmax(dim=1)  # Find max in columns 

    def forward(self, x):
        x = self.lin1(x)
        # print(x)
        x = self.relu(x)
        # print(x)
        x = self.lin2(x)
        # print(x)
        return x
    
mulklassify = MulClassify(6, 3, 10)

In [73]:
input_features = 6
num_class = 3

mulclass_x = torch.randn(100, input_features, dtype=torch.float32)
mulclass_y = torch.randint(0, input_features,(100, 1), dtype=torch.float32)

In [74]:
mulclass_y.type()

'torch.FloatTensor'

In [58]:
mulclass_y[0:5]

tensor([[2.],
        [1.],
        [1.],
        [0.],
        [3.]])

In [75]:
from torch.utils.data import DataLoader, Dataset
import numpy as np

class Mulcset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.samples = self.x.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.samples

In [76]:
mulc_set = Mulcset(x=mulclass_x,
                   y=mulclass_y)
mulc_set[1]

(tensor([ 0.0858, -0.4547, -0.0512, -0.3322,  0.3472, -0.4475]), tensor([0.]))

In [77]:
mulc_loader = DataLoader(mulc_set, 3)
iter_clas = iter(mulc_loader)
next(iter_clas)

[tensor([[-0.2776, -1.9314,  0.0392, -1.0306,  1.2299, -0.8924],
         [ 0.0858, -0.4547, -0.0512, -0.3322,  0.3472, -0.4475],
         [ 0.4138, -1.6741, -1.4273, -1.1846,  1.0657, -1.1606]]),
 tensor([[3.],
         [0.],
         [1.]])]

In [78]:
mulklassify(next(iter_clas)[0])

tensor([[0.0405, 0.4152, 0.0515],
        [0.2051, 0.4341, 0.0258],
        [0.1530, 0.2929, 0.0247]], grad_fn=<AddmmBackward0>)

In [63]:
out = mulklassify(next(iter_clas)[0])
print(out.shape)
out.argmax(dim=1)

torch.Size([3, 3])


tensor([2, 1, 2])

In [71]:
pred = mulklassify(next(iter_clas)[0])
print("Pred is: ", pred.argmax(dim=1))
print("Actual is:", next(iter_clas)[1])

Pred is:  tensor([1, 1, 1])
Actual is: tensor([[0.],
        [1.],
        [3.]])


In [ ]:
fn = nn.CrossEntropyLoss()
pred = mulklassify(next(iter_clas)[0])
# pred = pred.reshape(1, -1)
# pred = pred.argmax(dim=1)
print(pred)
# x = next(iter_clas)[1].squeeze()
# print(x)
loss = fn(pred, next(iter_clas)[1])
loss

In [83]:
# criterion = nn.CrossEntropyLoss()
fn = nn.CrossEntropyLoss()  # Binary Cross entropy had to be used.
optimiser = optim.SGD(params=mulklassify.parameters(), lr=0.01)

In [ ]:
# 100 pass through the full data using dataloader
for ep in range(100):
    for ind, batch in enumerate(mulc_loader):
        # print(batch[0].shape)  # torch.size([3, 4])
        pred_batch  = mulklassify(batch[0])
        loss_cr = fn(pred_batch, batch[1])
        # print(loss_cr.item())
        loss_cr.backward()  # backward pass happens with ease, even though there are multiple elements
        optimiser.step()
        optimiser.zero_grad()
    if ep % 5 == 0:
        print(f"Epoch is {ep} and loss is: {loss_cr.item():.2f}")

In [23]:
with torch.no_grad():
    test_pred = mulklassify([10][0])
    print(test_pred)
    print("compared to: ", mulc_set[10][1])

tensor([0.0012])
compared to:  tensor([0.])
